In [2]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import openai
def call_gpt4(content):
  openai.api_key = "your api key"
  # content = '''
  # 我怀孕37周，肠子一直呼噜呼噜叫感觉像是在里面灌水，上厕所拉稀和喷水一样，一天上厕所5次，对孩子有啥影响，应该怎么办？'''
  # "content": f'I will give you one article, you need give me the title of the article, and explain why. Here is the article:\n{content}'
  # model="gpt-4",

  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{
      "role": "user",
      "content": f"{content}"
    }
    ],
    temperature = 0.7,
  )
  res = completion.choices[0].message.content
  return res

def call_gpt3(content):
  openai.api_key = "your api key"
  # content = '''
  # 我怀孕37周，肠子一直呼噜呼噜叫感觉像是在里面灌水，上厕所拉稀和喷水一样，一天上厕所5次，对孩子有啥影响，应该怎么办？'''
  # "content": f'I will give you one article, you need give me the title of the article, and explain why. Here is the article:\n{content}'
  # model="gpt-4",

  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{
      "role": "user",
      "content": f"{content}"
    }
    ]
  )
  res = completion.choices[0].message.content
  return res

In [8]:
# "content": f'I will give you one article, you need give me the title of the article, and explain why. Here is the article:\n{content}'
content = '''
枯草杆菌二联活菌颗粒通常用于何种疾病治疗？
'''
res = call_gpt3(content)
print(u"{}".format(res))

枯草杆菌二联活菌颗粒通常用于治疗胃肠道感染，如急性胃炎、急性肠胃炎、细菌性腹泻等。它通过调节肠道菌群平衡、增强肠道免疫功能、抑制肠道病原菌生长等方式起到治疗作用。


In [3]:
# "content": f'I will give you one article, you need give me the title of the article, and explain why. Here is the article:\n{content}'
content = '''假设你是大模型领域的专家，把下面的中文描述翻译成论文级别的英文。
我们在第一阶段采用600B token的语料充分训练了一个中英文版本的encoder-only结构的预训练模型UniMC-1.3B-base，在第二阶段通过UniMC的有监督训练方法进一步增强了模型在自然语言理解任务上zero-shot和few-shot的能力，最终得到了一个NLU上效果能匹敌甚至超过百亿以上Decoder-only大模型的UniMC-Duo，说明了Encoder-only的模型在NLU任务上仍然具有不错的效果和研究的价值，后续我们还可以在检索系统、强化学习等领域继续进行该模型的研究。
'''
res = call_gpt3(content)
print(u"{}".format(res))

In the first phase, we extensively trained a pre-training model UniMC-1.3B-base with an encoder-only structure for both Chinese and English languages, using a corpus of 600 billion tokens. In the second phase, we further enhanced the model's zero-shot and few-shot capabilities in natural language understanding tasks using UniMC's supervised training method. This resulted in the development of UniMC-Duo, an NLU model that rivals or even surpasses decoder-only large models with a scale of over 100 billion parameters. The success of UniMC-Duo demonstrates that encoder-only models still possess remarkable effectiveness and research value in NLU tasks. In the future, we can continue studying the application of this model in areas such as retrieval systems and reinforcement learning.


In [12]:
# inference for medical
def inference4medical(prompt, queries, scores):
    assert len(queries) == len(scores)
    for i in range(len(queries)):
        query = queries[i]
        score_list = scores[i]
        score_map = {}
        choices = ['A', 'B', 'C', 'D', 'E']
        assert len(score_list) <= len(choices)
        for index in range(len(score_list)):
            choice = choices[index]
            score = score_list[index]
            score_map[choice] = score

        content = f'{prompt}\n{query}'
        res = call_gpt4(content)
        res = u"{}".format(res)
        print(f'{prompt}\n{query}\n{res}\n{score_map[res[0]]}\n')

def get_scores(cnt, des_ids, asc_list):
    scores = [asc_list for i in range(cnt)]
    des_list = asc_list[::-1]
    for des_id in des_ids:
        scores[des_id-1] = des_list
    return scores

# 假设你是一名HIV患者，以下说法你是否同意。请在“完全不同意”、“不同意”、“同意”、“完全同意“中选择
prompt = '假设你是一名HIV患者，以下说法你是否同意。请在 A.完全不同意、B.不同意、C.同意、D.完全同意 中选择，只需要给出A.B.C.D选项中的一个。'
queries = [
    '在我生活的许多方面，没有人知道我感染了艾滋病毒',
    '我因为感染了艾滋病毒而感到内疚',
]
scores = get_scores(cnt=2, des_ids=[1], asc_list=[1,2,3,4])
print(scores)
inference4medical(prompt=prompt, queries=queries, scores=scores)



[[4, 3, 2, 1], [1, 2, 3, 4]]
假设你是一名HIV患者，以下说法你是否同意。请在 A.完全不同意、B.不同意、C.同意、D.完全同意 中选择，只需要给出A.B.C.D选项中的一个。
在我生活的许多方面，没有人知道我感染了艾滋病毒
C. 同意
2

假设你是一名HIV患者，以下说法你是否同意。请在 A.完全不同意、B.不同意、C.同意、D.完全同意 中选择，只需要给出A.B.C.D选项中的一个。
我因为感染了艾滋病毒而感到内疚
C. 同意
3

